In [5]:
import requests
import json

# URL de la API
url = "https://apps.sec.cl/INTONLINEv1/ClientesAfectados/GetPorFecha"

# Headers (debes copiar los mismos headers que usa el navegador si es necesario)
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Origin": "https://apps.sec.cl",
    "Referer": "https://apps.sec.cl/INTONLINEv1/index.aspx",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
}

# Payload con la fecha y la hora que quieres consultar
payload = {
    "anho": 2025,
    "mes": 1,
    "dia": 29,
    "hora": 12
}

# Hacer la solicitud POST
response = requests.post(url, headers=headers, json=payload)

# Revisar si la respuesta es exitosa (código 200)
if response.status_code == 200:
    # Convertir la respuesta JSON en un diccionario de Python
    data = response.json()
    
    # Guardar en un archivo JSON para luego cargarlo en Power BI
    with open("clientes_afectados.json", "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
    
    print("Datos guardados en clientes_afectados.json")
else:
    print(f"Error en la solicitud: {response.status_code}")
    print(response.text)  # Muestra el mensaje de error si lo hay


Datos guardados en clientes_afectados.json


In [6]:
import requests
import json
import pandas as pd
from datetime import datetime

# Configurar URL y payload
url = "https://apps.sec.cl/INTONLINEv1/ClientesAfectados/GetPorFecha"
payload = {"anho": 2025, "mes": 1, "dia": 29, "hora": 12}  # Ajusta con la hora actual
headers = {"Content-Type": "application/json"}

# Hacer la solicitud POST
response = requests.post(url, json=payload, headers=headers)

# Procesar respuesta
if response.status_code == 200:
    data = response.json()

    # Agregar un timestamp real
    for entry in data:
        entry["TIMESTAMP"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Guardar en un archivo CSV
    df = pd.DataFrame(data)
    df.to_csv("clientes_afectados.csv", mode="a", index=False, header=False)  # Append

    print("Datos guardados correctamente")

else:
    print("Error en la solicitud:", response.status_code)


Datos guardados correctamente


In [14]:
import requests
import json
import pandas as pd
import os
from datetime import datetime, timedelta

# Configurar URL y payload
url = "https://apps.sec.cl/INTONLINEv1/ClientesAfectados/GetPorFecha"
payload = {"anho": 2025, "mes": 1, "dia": 29, "hora": 12}  # Ajusta la hora actual
headers = {"Content-Type": "application/json"}

# Obtener el timestamp exacto en este momento
timestamp_actual = datetime.now()

# Hacer la solicitud POST
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    try:
        data = response.json()

        # Lista para almacenar los datos procesados
        registros = []

        for entry in data:
            actualizado_hace = entry.get("ACTUALIZADO_HACE", "")
            minutos_atras = 0  

            try:
                partes = actualizado_hace.split(" ")
                for i, part in enumerate(partes):
                    if part == "Minutos" and i > 0:
                        minutos_atras = int(partes[i-1])  
            except Exception as e:
                print("Error al extraer minutos de ACTUALIZADO_HACE:", e)

            # Calcular la hora exacta del corte
            hora_corte = timestamp_actual - timedelta(minutes=minutos_atras)

            # Asignar timestamps correctos
            entry["HORA_CORTE"] = hora_corte.strftime("%Y-%m-%d %H:%M:%S")
            entry["TIMESTAMP"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Se asegura que cada fila tenga la hora real exacta

            registros.append(entry)

        # Convertir a DataFrame
        df = pd.DataFrame(registros)

        # Nombre del archivo CSV
        file_name = "clientes_afectados.csv"

        # Guardar en CSV con encabezados si no existe, o sin encabezados si ya existe
        file_exists = os.path.exists(file_name)
        df.to_csv(file_name, mode="a", index=False, header=not file_exists)

        print(f"Datos guardados correctamente en {file_name}")

    except json.JSONDecodeError:
        print("Error: La respuesta no es un JSON válido.")
else:
    print("Error en la solicitud:", response.status_code)


Datos guardados correctamente en clientes_afectados.csv


In [22]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# Configurar URL y payload
url = "https://apps.sec.cl/INTONLINEv1/ClientesAfectados/GetPorFecha"
payload = {"anho": 2025, "mes": 1, "dia": 29, "hora": 19}
headers = {"Content-Type": "application/json"}

# Hacer la solicitud POST
response = requests.post(url, json=payload, headers=headers)

if response.status_code == 200:
    data = response.json()

    # Agregar timestamp real de la consulta
    timestamp_actual = datetime.now()

    # Procesar datos en una lista
    registros = []
    for entry in data:
        actualizado_hace = entry.get("ACTUALIZADO_HACE", "")
        minutos_atras = 0

        if "Minutos" in actualizado_hace:
            partes = actualizado_hace.split(" ")
            for i, part in enumerate(partes):
                if part == "Minutos":
                    minutos_atras = int(partes[i-1])

        # Calcular la hora exacta del corte
        hora_corte = timestamp_actual - timedelta(minutes=minutos_atras)
        entry["HORA_CORTE"] = hora_corte.strftime("%Y-%m-%d %H:%M:%S")
        entry["TIMESTAMP"] = timestamp_actual.strftime("%Y-%m-%d %H:%M:%S")

        registros.append(entry)

    # Convertir a DataFrame
    df_new = pd.DataFrame(registros)

    # Cargar datos previos si existen
    file_name = "clientes_afectados.csv"
    try:
        df_prev = pd.read_csv(file_name)
    except FileNotFoundError:
        df_prev = pd.DataFrame()

    # Unir los datos previos con los nuevos
    df_total = pd.concat([df_prev, df_new])

    # Eliminar duplicados manteniendo solo la última entrada por fecha y comuna
    df_total = df_total.drop_duplicates(subset=["FECHA_INT_STR", "NOMBRE_COMUNA"], keep="last")

    # Guardar el CSV limpio
    df_total.to_csv(file_name, index=False)

    print("Datos guardados correctamente sin duplicados.")

else:
    print("Error en la solicitud:", response.status_code)


Datos guardados correctamente sin duplicados.


In [25]:
from playwright.sync_api import sync_playwright
import json
import pandas as pd
import os

# Archivo donde se guardarán los datos
csv_file = "clientes_afectados_playwright.csv"

# Función para capturar respuestas de la API y guardarlas en CSV
def intercept_responses():
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)  # Ejecutar sin abrir ventana
        page = browser.new_page()

        # Lista para almacenar datos antes de guardarlos
        registros = []

        # Interceptar respuestas de la API
        def handle_response(response):
            if "GetPorFecha" in response.url:
                try:
                    data = response.json()
                    for entry in data:
                        # Extraer los datos de la respuesta
                        registros.append({
                            "FECHA": entry.get("FECHA_INT_STR", ""),
                            "REGION": entry.get("NOMBRE_REGION", ""),
                            "COMUNA": entry.get("NOMBRE_COMUNA", ""),
                            "EMPRESA": entry.get("NOMBRE_EMPRESA", ""),
                            "CLIENTES_AFECTADOS": entry.get("CLIENTES_AFECTADOS", 0),
                            "ACTUALIZADO_HACE": entry.get("ACTUALIZADO_HACE", ""),
                        })
                except:
                    print("❌ No se pudo leer la respuesta JSON")

        # Escuchar respuestas
        page.on("response", handle_response)

        # Abrir la página web
        page.goto("https://apps.sec.cl/INTONLINEv1/index.aspx")

        # Esperar unos segundos para que se capturen los datos
        page.wait_for_timeout(5000)

        # Cerrar el navegador
        browser.close()

        # Guardar en CSV
        if registros:
            df = pd.DataFrame(registros)

            # Guardar en CSV (añadiendo si ya existe)
            if not os.path.exists(csv_file):
                df.to_csv(csv_file, index=False, mode="w", encoding="utf-8-sig")
            else:
                df.to_csv(csv_file, index=False, mode="a", encoding="utf-8-sig", header=False)

            print(f"✅ Datos guardados correctamente en {csv_file}")

# Ejecutar la función
intercept_responses()


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.